This notebook implements the Auto-G Computation algorithm

In [520]:
import sys
sys.path.append('..')
sys.path.append('.')

In [527]:
from main import util
from main import data_generator as dg
from main import maximal_independent_set
import numpy as np
import pandas as pd
import random
import os
import pickle
from scipy.optimize import minimize
from scipy.integrate import quad
import math

In [602]:
def df_for_estimation(network, ind_set, sample):
    '''Function to assemble data to estimate the distribution f(y | -) and f(l |-)'''
    
    data_list = []
    
    for i in ind_set['subject']:
        # extract the relevant rows from sample based on index i. 
        row = sample.loc[i]
        
        # assuming 'Y', 'A', and 'L' are column names in 'sample'
        l_i = row['L']
        a_i = row['A']
        y_i = row['Y']
        
        # get the neighbors of i
        N_i = util.kth_order_neighborhood(network, i, 1)
        normalizing_weight = 1 #/ len(N_i)
        
        # Get a list with sample.loc[j]['Y'] for each j in N_i
        l_j = [sample.loc[j]['L'] for j in N_i]
        a_j = [sample.loc[j]['A'] for j in N_i]
        y_j = [sample.loc[j]['Y'] for j in N_i]
        
        # sum over l_j and then mutiply with normalizing_weight
        sum_l_j = sum(l_j)
        adjusted_sum_l_j = sum_l_j * normalizing_weight

        sum_a_j = sum(a_j)
        adjusted_sum_a_j = sum_a_j * normalizing_weight
        
        w_ij_y = normalizing_weight
        w_ij_l = normalizing_weight
        
        data_list.append({
            'i' : i,
            'y_i': y_i,
            'a_i': a_i,
            'l_i': l_i,
            'l_j_list': l_j,
            'y_j_list': y_j,  
            'adjusted_sum_l_j': adjusted_sum_l_j,
            'adjusted_sum_a_j': adjusted_sum_a_j,
            'w_ij_y': w_ij_yw_ij_theta,
            'w_ij_l': w_ij_l 
        })
    
    df = pd.DataFrame(data_list) 
    return df     

In [603]:
# sketch

# get network (UUU) data
# find maximal_1_apart_independent_set
# assemble Y_est_data using networkdata and indset

# specify the form of the density f(yi | -)
# specify the negative log-likelihood function
# use scipy.optimize minimize to estmate tau_y
# now plug tau_y hat back in we will get f(y | -)

In [579]:
### FOR AUTO LOGISTIC MODEL ###


### L ###
def H_i(l_i, adjusted_sum_l_j, params):
    beta_0, beta_1 = params
    return beta_0 + beta_1*adjusted_sum_l_j

def W_i(data_i, params, l_i=None):
    # we allow the caller to pass in l_i separate from data_i
    # so that when evaluating the denominator it is easier.
    if l_i == None:
        l_i = data_i['l_i']
    H_i_output = H_i(l_i=l_i, 
                     adjusted_sum_l_j=data_i['adjusted_sum_l_j'], 
                     params=params)
    
    second_term = 0
    for l_j in data_i['l_j_list']:
        second_term += l_i * l_j * data_i['w_ij_l']
    
    return l_i * H_i_output + second_term

def f_L_i_given_stuff(data_i, params):
    numerator = math.exp(W_i(data_i, params))
    
    denominator = 0
    for l_i_val in [0, 1]: # sum over all possible values of l_i (which is 1 and 0 cuz y is binary)
        denominator += math.exp(W_i(data_i, params, l_i=l_i_val))
    
    return numerator / denominator


### Y ###
def G_i(y_i, a_i, l_i, adjusted_sum_a_j, adjusted_sum_l_j, params):
    beta_0, beta_1, beta_2, beta_3, beta_4 = params[:-1]
    return beta_0 + beta_1*a_i + beta_2*l_i + beta_3*adjusted_sum_a_j + beta_4*adjusted_sum_l_j

def theta_ij(w_ij_y, params):
    theta = params[-1]
    return w_ij_y * theta

def U_i(data_i, params, y_i=None):
    # we allow the caller to pass in y_i separate from data_i
    # so that when evaluating the denominator it is easier.
    if y_i == None:
        y_i = data_i['y_i']
    G_i_output = G_i(y_i=y_i, 
                     a_i=data_i['a_i'], 
                     l_i=data_i['l_i'], 
                     adjusted_sum_a_j=data_i['adjusted_sum_a_j'], 
                     adjusted_sum_l_j=data_i['adjusted_sum_l_j'], 
                     params=params)
    
    second_term = 0
    for y_j in data_i['y_j_list']:
        second_term += y_i * y_j * theta_ij(data_i['w_ij_y'], params)
    
    return y_i * G_i_output + second_term

def f_Y_i_given_stuff(data_i, params):
    numerator = math.exp(U_i(data_i, params))
    
    denominator = 0
    for y_i_val in [0, 1]: # sum over all possible values of y_i (which is 1 and 0 cuz y is binary)
        denominator += math.exp(U_i(data_i, params, y_i=y_i_val))
    
    return numerator / denominator

In [588]:
def nlcl(params, f, est_df):
    '''
    negative log coding likelihood
    f: a function, either f_Y_i_given_stuff or f_L_i_given_stuff
    '''
    log_likelihoods = est_df.apply(lambda row: np.log(f(row, params)), axis=1)
    return -np.sum(log_likelihoods)

def optimize_params(nll_function, initial_params, f, est_df):
    result = minimize(nll_function, x0=initial_params, args=(f, est_df))
    return result.x

In [581]:
def gibbs_sampler_1(n_samples, burn_in, network, f_Yi, f_Li, verbose, A_val):
    '''
    This implementation closely follows the description on page 10 of the auto-g paper.
    '''
    samples = []
    N = len(network) # number of subjects in the network
    
    # produce initial random guess
    sample = pd.DataFrame(index=network.keys(), columns=['L', 'A', 'Y'])
    sample['L'] = {vertex: random.choice([1, 0]) for vertex in network.keys()}
    sample['A'] = {vertex: A_val for vertex in network.keys()}
    sample['Y'] = {vertex: random.choice([1, 0]) for vertex in network.keys()}

    for m in range(n_samples + burn_in):
        if verbose and m % 1000 == 0:
            print("progress: ", m / (n_samples + burn_in))
            
        i = (m % N) # the paper has "+1" but i don't +1 because the index of subjects in my network starts from 0
        # draw L_i ~ f(L_i | L_-i)
        boundary_values_L = {
            'L_neighbors': [sample.loc[neighbor, 'L'] for neighbor in network[i]]
        }
        new_Li = draw(dist=f_Li, inputs=boundary_values_L)
        
        # draw Y_i ~ f(Y_i | Y_-i)
        boundary_values_Y = {
            'L_self': sample.loc[i, 'L'],
            'L_neighbors': [sample.loc[neighbor, 'L'] for neighbor in network[i]],
            'A_self': sample.loc[i, 'A'],
            'A_neighbors': [sample.loc[neighbor, 'A'] for neighbor in network[i]],
            'Y_neighbors': [sample.loc[neighbor, 'Y'] for neighbor in network[i]],
        }
        new_Yi = draw(dist=f_Yi, inputs=boundary_values_Y)

        sample.loc[i, 'L'] = new_Li
        sample.loc[i, 'Y'] = new_Yi

        if m >= burn_in:
            samples.append(sample.copy())

    return samples

In [582]:
def beta_i_a(samples, i, select_for_every):
    '''
    samples: samples from gibbs_sampler_1
    select_for_every: compute the causal effect using samples for every select_for_every iterations 
                      to "thin" autocorrelation
    '''
    
    selected_Y_values = []
    
    # Iterate through the samples, selecting every nth sample where n is select_for_every
    for sample_idx in range(0, len(samples), select_for_every):
        selected_sample = samples[sample_idx]
        selected_Y_values.append(selected_sample.loc[i, 'Y'])
        
    average_Y = float(sum(selected_Y_values) / len(selected_Y_values))
    return average_Y

In [583]:
def draw(dist, inputs):
    proba_of_1 = dist(inputs)
    return np.random.binomial(1, proba_of_1)

In [613]:
def sample_given_boundary_binary(boundary_values):
    '''
    Note: This can't be any random function. 
          Check Lauritzen chain graph paper page 342.
    '''
    weighted_sum = 0
    weights = {
        'Y_neighbors': -0.1, # this need to be controlled
        'L_self': 0.8,
        'A_self': 1.7,
        'L_neighbors': -0.1, # this need to be controlled
        'A_neighbors': -0.1 # this need to be controlled
    }
    
    for key, values in boundary_values.items():
        if values is not None and values != []:
            if isinstance(values, list):
                weighted_sum += weights[key] * sum(values)
            else:
                weighted_sum += weights[key] * values

    noise = np.random.normal(0, 0.1)
    p = expit(weighted_sum + noise)
    return p

In [614]:
# steps to test for the correctness of auto g

# generate a small network n
network = util.create_random_network(n=1000, min_neighbors=1, max_neighbors=6)

# evaluate true causal effect using true f_Yi, true f_Li, n, and A_val (via beta_i_a and gibbs sampler 1)
true_f_Yi = sample_given_boundary_binary
true_f_Li = sample_given_boundary_binary

Y_A1 = gibbs_sampler_1(n_samples=40000, burn_in=10000, network=network, f_Yi=true_f_Yi, f_Li=true_f_Li, 
                       verbose=True, A_val=1)

Y_A0 = gibbs_sampler_1(n_samples=40000, burn_in=10000, network=network, f_Yi=true_f_Yi, f_Li=true_f_Li, 
                       verbose=True, A_val=0)

# generate a UUU realization based on the structure of n
# estmate f_Yi hat, f_Li hat using auto_g & network realization
# estimate causal effect using f_Yi hat, f_Li hat, n, and A_val (via beta_i_a and gibbs sampler 1)

progress:  0.0


/var/folders/2m/s6xvfp8x29j9m9pl6w7jn8hh0000gn/T/ipykernel_56562/2532712610.py:16: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if values is not None and values != []:


progress:  0.02
progress:  0.04
progress:  0.06
progress:  0.08
progress:  0.1
progress:  0.12
progress:  0.14
progress:  0.16
progress:  0.18
progress:  0.2
progress:  0.22
progress:  0.24
progress:  0.26
progress:  0.28
progress:  0.3
progress:  0.32
progress:  0.34
progress:  0.36
progress:  0.38
progress:  0.4
progress:  0.42
progress:  0.44
progress:  0.46
progress:  0.48
progress:  0.5
progress:  0.52
progress:  0.54
progress:  0.56
progress:  0.58
progress:  0.6
progress:  0.62
progress:  0.64
progress:  0.66
progress:  0.68
progress:  0.7
progress:  0.72
progress:  0.74
progress:  0.76
progress:  0.78
progress:  0.8
progress:  0.82
progress:  0.84
progress:  0.86
progress:  0.88
progress:  0.9
progress:  0.92
progress:  0.94
progress:  0.96
progress:  0.98
progress:  0.0


/var/folders/2m/s6xvfp8x29j9m9pl6w7jn8hh0000gn/T/ipykernel_56562/2532712610.py:16: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if values is not None and values != []:


progress:  0.02
progress:  0.04
progress:  0.06
progress:  0.08
progress:  0.1
progress:  0.12
progress:  0.14
progress:  0.16
progress:  0.18
progress:  0.2
progress:  0.22
progress:  0.24
progress:  0.26
progress:  0.28
progress:  0.3
progress:  0.32
progress:  0.34
progress:  0.36
progress:  0.38
progress:  0.4
progress:  0.42
progress:  0.44
progress:  0.46
progress:  0.48
progress:  0.5
progress:  0.52
progress:  0.54
progress:  0.56
progress:  0.58
progress:  0.6
progress:  0.62
progress:  0.64
progress:  0.66
progress:  0.68
progress:  0.7
progress:  0.72
progress:  0.74
progress:  0.76
progress:  0.78
progress:  0.8
progress:  0.82
progress:  0.84
progress:  0.86
progress:  0.88
progress:  0.9
progress:  0.92
progress:  0.94
progress:  0.96
progress:  0.98


In [615]:
ce = [beta_i_a(Y_A1, i, 3) - beta_i_a(Y_A0, i, 3) for i in range(0, 1000)]

In [616]:
np.mean(ce)

-0.0722229638518074

In [514]:
def generate_edge_types(true_model):
    # Build edge_types based on edge types at each layer
    edge_types = {}

    if true_model[0] == 'U': 
        edge_types['L'] = ['U', {'sample_given_boundary':dg.sample_given_boundary_binary, 'verbose':VERBOSE, 'burn_in':BURN_IN}]
    else:
        edge_types['L'] = ['B', {'U_dist':dg.U_dist_1, 'f':dg.f_binary}]

    if true_model[1] == 'U': 
        edge_types['A'] = ['U', {'sample_given_boundary':dg.sample_given_boundary_binary, 'verbose':VERBOSE, 'burn_in':BURN_IN}]
    else:
        edge_types['A'] = ['B', {'U_dist':dg.U_dist_1, 'f':dg.f_binary}]

    if true_model[2] == 'U': 
        edge_types['Y'] = ['U', {'sample_given_boundary':dg.sample_given_boundary_binary, 'verbose':VERBOSE, 'burn_in':BURN_IN}]
    else:
        edge_types['Y'] = ['B', {'U_dist':dg.U_dist_1, 'f':dg.f_binary}]

    return edge_types

In [524]:
BURN_IN = 300
VERBOSE = True
edge_types = generate_edge_types("UUU")

# Sample a single realization from the specified Graphical Model
GM_sample = dg.sample_L_A_Y(n_samples=1, network=network, edge_types=edge_types)[0]

[PROGRESS] Sample from UG burning in: 0 / 300
[PROGRESS] Sample from UG burning in: 1 / 300
[PROGRESS] Sample from UG burning in: 2 / 300
[PROGRESS] Sample from UG burning in: 3 / 300
[PROGRESS] Sample from UG burning in: 4 / 300
[PROGRESS] Sample from UG burning in: 5 / 300
[PROGRESS] Sample from UG burning in: 6 / 300
[PROGRESS] Sample from UG burning in: 7 / 300
[PROGRESS] Sample from UG burning in: 8 / 300
[PROGRESS] Sample from UG burning in: 9 / 300
[PROGRESS] Sample from UG burning in: 10 / 300
[PROGRESS] Sample from UG burning in: 11 / 300
[PROGRESS] Sample from UG burning in: 12 / 300
[PROGRESS] Sample from UG burning in: 13 / 300
[PROGRESS] Sample from UG burning in: 14 / 300
[PROGRESS] Sample from UG burning in: 15 / 300
[PROGRESS] Sample from UG burning in: 16 / 300
[PROGRESS] Sample from UG burning in: 17 / 300
[PROGRESS] Sample from UG burning in: 18 / 300
[PROGRESS] Sample from UG burning in: 19 / 300
[PROGRESS] Sample from UG burning in: 20 / 300
[PROGRESS] Sample from 

[PROGRESS] Sample from UG burning in: 196 / 300
[PROGRESS] Sample from UG burning in: 197 / 300
[PROGRESS] Sample from UG burning in: 198 / 300
[PROGRESS] Sample from UG burning in: 199 / 300
[PROGRESS] Sample from UG burning in: 200 / 300
[PROGRESS] Sample from UG burning in: 201 / 300
[PROGRESS] Sample from UG burning in: 202 / 300
[PROGRESS] Sample from UG burning in: 203 / 300
[PROGRESS] Sample from UG burning in: 204 / 300
[PROGRESS] Sample from UG burning in: 205 / 300
[PROGRESS] Sample from UG burning in: 206 / 300
[PROGRESS] Sample from UG burning in: 207 / 300
[PROGRESS] Sample from UG burning in: 208 / 300
[PROGRESS] Sample from UG burning in: 209 / 300
[PROGRESS] Sample from UG burning in: 210 / 300
[PROGRESS] Sample from UG burning in: 211 / 300
[PROGRESS] Sample from UG burning in: 212 / 300
[PROGRESS] Sample from UG burning in: 213 / 300
[PROGRESS] Sample from UG burning in: 214 / 300
[PROGRESS] Sample from UG burning in: 215 / 300
[PROGRESS] Sample from UG burning in: 21

[PROGRESS] Sample from UG burning in: 72 / 300
[PROGRESS] Sample from UG burning in: 73 / 300
[PROGRESS] Sample from UG burning in: 74 / 300
[PROGRESS] Sample from UG burning in: 75 / 300
[PROGRESS] Sample from UG burning in: 76 / 300
[PROGRESS] Sample from UG burning in: 77 / 300
[PROGRESS] Sample from UG burning in: 78 / 300
[PROGRESS] Sample from UG burning in: 79 / 300
[PROGRESS] Sample from UG burning in: 80 / 300
[PROGRESS] Sample from UG burning in: 81 / 300
[PROGRESS] Sample from UG burning in: 82 / 300
[PROGRESS] Sample from UG burning in: 83 / 300
[PROGRESS] Sample from UG burning in: 84 / 300
[PROGRESS] Sample from UG burning in: 85 / 300
[PROGRESS] Sample from UG burning in: 86 / 300
[PROGRESS] Sample from UG burning in: 87 / 300
[PROGRESS] Sample from UG burning in: 88 / 300
[PROGRESS] Sample from UG burning in: 89 / 300
[PROGRESS] Sample from UG burning in: 90 / 300
[PROGRESS] Sample from UG burning in: 91 / 300
[PROGRESS] Sample from UG burning in: 92 / 300
[PROGRESS] Sa

[PROGRESS] Sample from UG burning in: 246 / 300
[PROGRESS] Sample from UG burning in: 247 / 300
[PROGRESS] Sample from UG burning in: 248 / 300
[PROGRESS] Sample from UG burning in: 249 / 300
[PROGRESS] Sample from UG burning in: 250 / 300
[PROGRESS] Sample from UG burning in: 251 / 300
[PROGRESS] Sample from UG burning in: 252 / 300
[PROGRESS] Sample from UG burning in: 253 / 300
[PROGRESS] Sample from UG burning in: 254 / 300
[PROGRESS] Sample from UG burning in: 255 / 300
[PROGRESS] Sample from UG burning in: 256 / 300
[PROGRESS] Sample from UG burning in: 257 / 300
[PROGRESS] Sample from UG burning in: 258 / 300
[PROGRESS] Sample from UG burning in: 259 / 300
[PROGRESS] Sample from UG burning in: 260 / 300
[PROGRESS] Sample from UG burning in: 261 / 300
[PROGRESS] Sample from UG burning in: 262 / 300
[PROGRESS] Sample from UG burning in: 263 / 300
[PROGRESS] Sample from UG burning in: 264 / 300
[PROGRESS] Sample from UG burning in: 265 / 300
[PROGRESS] Sample from UG burning in: 26

[PROGRESS] Sample from UG burning in: 119 / 300
[PROGRESS] Sample from UG burning in: 120 / 300
[PROGRESS] Sample from UG burning in: 121 / 300
[PROGRESS] Sample from UG burning in: 122 / 300
[PROGRESS] Sample from UG burning in: 123 / 300
[PROGRESS] Sample from UG burning in: 124 / 300
[PROGRESS] Sample from UG burning in: 125 / 300
[PROGRESS] Sample from UG burning in: 126 / 300
[PROGRESS] Sample from UG burning in: 127 / 300
[PROGRESS] Sample from UG burning in: 128 / 300
[PROGRESS] Sample from UG burning in: 129 / 300
[PROGRESS] Sample from UG burning in: 130 / 300
[PROGRESS] Sample from UG burning in: 131 / 300
[PROGRESS] Sample from UG burning in: 132 / 300
[PROGRESS] Sample from UG burning in: 133 / 300
[PROGRESS] Sample from UG burning in: 134 / 300
[PROGRESS] Sample from UG burning in: 135 / 300
[PROGRESS] Sample from UG burning in: 136 / 300
[PROGRESS] Sample from UG burning in: 137 / 300
[PROGRESS] Sample from UG burning in: 138 / 300
[PROGRESS] Sample from UG burning in: 13

[PROGRESS] Sample from UG burning in: 290 / 300
[PROGRESS] Sample from UG burning in: 291 / 300
[PROGRESS] Sample from UG burning in: 292 / 300
[PROGRESS] Sample from UG burning in: 293 / 300
[PROGRESS] Sample from UG burning in: 294 / 300
[PROGRESS] Sample from UG burning in: 295 / 300
[PROGRESS] Sample from UG burning in: 296 / 300
[PROGRESS] Sample from UG burning in: 297 / 300
[PROGRESS] Sample from UG burning in: 298 / 300
[PROGRESS] Sample from UG burning in: 299 / 300


In [604]:
ind_set = maximal_independent_set.maximal_n_apart_independent_set(network, 1)
ind_set = pd.DataFrame(list(ind_set), columns=["subject"])
est_df = df_for_estimation(network=network, ind_set=ind_set, sample=GM_sample)

In [617]:
beta_0, beta_1, beta_2, beta_3, beta_4, theta = [0.5]*6
initial_params = [beta_0, beta_1, beta_2, beta_3, beta_4, theta]
params_Y = optimize_params(nlcl, initial_params, f_Y_i_given_stuff, est_df)

beta_0, beta_1 = [0]*2
initial_params = [beta_0, beta_1]
params_L = optimize_params(nlcl, initial_params, f_L_i_given_stuff, est_df)

In [618]:
params_Y

array([-0.42806528, -0.18069905,  0.22173799, -0.01905073,  0.03608502,
        0.03708152])

In [619]:
def hat_f_Yi(inputs):
    '''
    return: proba of yi = 1
    '''
    y_i = 1
    l_i = inputs['L_self']
    a_i = inputs['A_self']
        
    normalizing_weight = 1 #/ len(inputs['Y_neighbors'])
        
    l_j = inputs['L_neighbors']
    a_j = inputs['A_neighbors']
    y_j = inputs['Y_neighbors']
    
    sum_l_j = sum(l_j)
    adjusted_sum_l_j = sum_l_j * normalizing_weight

    sum_a_j = sum(a_j)
    adjusted_sum_a_j = sum_a_j * normalizing_weight

    w_ij_y = normalizing_weight
    w_ij_l = normalizing_weight
        
    inputs = {
        'y_i': y_i,
        'a_i': a_i,
        'l_i': l_i,
        'l_j_list': l_j,
        'y_j_list': y_j,  
        'adjusted_sum_l_j': adjusted_sum_l_j,
        'adjusted_sum_a_j': adjusted_sum_a_j,
        'w_ij_y': w_ij_yw_ij_theta,
        'w_ij_l': w_ij_l 
    }
        
    return f_Y_i_given_stuff(inputs, params_Y)


def hat_f_Li(inputs):
    '''
    return: proba of yi = 1
    
    ata_i['l_i']
    H_i_output = H_i(l_i=l_i, 
                     adjusted_sum_l_j=data_i['adjusted_sum_l_j'], 
                     params=params)
    
    second_term = 0
    for l_j in data_i['l_j_list']:
        second_term += l_i * l_j * data_i['w_ij_l']
    '''
    l_i = 1
        
    normalizing_weight = 1 #/ len(inputs['L_neighbors'])
        
    l_j = inputs['L_neighbors']
    
    sum_l_j = sum(l_j)
    adjusted_sum_l_j = sum_l_j * normalizing_weight

    w_ij_l = normalizing_weight
        
    inputs = {
        'l_i': l_i,
        'l_j_list': l_j,  
        'adjusted_sum_l_j': adjusted_sum_l_j,
        'w_ij_l': w_ij_l 
    }
        
    return f_L_i_given_stuff(inputs, params_L)

In [620]:

Y_A1 = gibbs_sampler_1(n_samples=40000, burn_in=10000, network=network, f_Yi=hat_f_Yi, f_Li=hat_f_Li, 
                       verbose=True, A_val=1)

Y_A0 = gibbs_sampler_1(n_samples=40000, burn_in=10000, network=network, f_Yi=hat_f_Yi, f_Li=hat_f_Li, 
                       verbose=True, A_val=0)



progress:  0.0
progress:  0.02
progress:  0.04
progress:  0.06
progress:  0.08
progress:  0.1
progress:  0.12
progress:  0.14
progress:  0.16
progress:  0.18
progress:  0.2
progress:  0.22
progress:  0.24
progress:  0.26
progress:  0.28
progress:  0.3
progress:  0.32
progress:  0.34
progress:  0.36
progress:  0.38
progress:  0.4
progress:  0.42
progress:  0.44
progress:  0.46
progress:  0.48
progress:  0.5
progress:  0.52
progress:  0.54
progress:  0.56
progress:  0.58
progress:  0.6
progress:  0.62
progress:  0.64
progress:  0.66
progress:  0.68
progress:  0.7
progress:  0.72
progress:  0.74
progress:  0.76
progress:  0.78
progress:  0.8
progress:  0.82
progress:  0.84
progress:  0.86
progress:  0.88
progress:  0.9
progress:  0.92
progress:  0.94
progress:  0.96
progress:  0.98
progress:  0.0
progress:  0.02
progress:  0.04
progress:  0.06
progress:  0.08
progress:  0.1
progress:  0.12
progress:  0.14
progress:  0.16
progress:  0.18
progress:  0.2
progress:  0.22
progress:  0.24
progr

In [621]:
ce = [beta_i_a(Y_A1, i, 3) - beta_i_a(Y_A0, i, 3) for i in range(0, 1000)]

In [622]:
np.mean(ce)

-0.05987370631468427

In [473]:
DATA_SOURCE = "../data/simulation_autog/"
TRUE_MODEL = "UUU"

In [135]:
# get network (UUU) data
network_path = os.path.join(DATA_SOURCE, 'network.pkl')
sample_data_path = os.path.join(DATA_SOURCE, f"{TRUE_MODEL}_sample.csv")
ind_set_path = os.path.join(DATA_SOURCE, '1_ind_set.csv')

with open(network_path, 'rb') as file:
    network = pickle.load(file)

GM_sample = pd.read_csv(sample_data_path)
ind_set = pd.read_csv(ind_set_path)

In [140]:
est_df = df_for_estimation(network=network, ind_set=ind_set, sample=GM_sample)

In [120]:
# Example from meeting with Rohit

from scipy.optimize import minimize
from scipy.special import expit

def f(x):
    return x[0]**2 + x[1]**2

guess = [-10, 10]
result = minimize(f, x0=guess, method="L-BFGS-B")
print(result.x)


X = np.random.normal(0, 1, 2000)
Y = np.random.binomial(1, expit(0.5 + 2*X), 2000)

def nll(theta, X, Y):
    pY1 = expit(theta[0] + theta[1]*X)
    pY = np.log(Y*pY1 + (1-Y)*(1-pY1))
    return -np.sum(pY)

result = minimize(nll, x0=np.random.uniform(-1, 1, 2), args=(X, Y))
print(result.x)

[-3.23724727e-07  3.23724727e-07]
[0.4628976  1.97511989]


In [ ]:
### FOR AUTO GAUSSIAN MODEL ###

def G_i(y_i, a_i, l_i, adjusted_sum_a_j, adjusted_sum_l_j, params):
    beta_0, betah_1, beta_2, beta_3, beta_4, sigma2_y = params[:-1]
    mu_y_i = beta_0 + beta_1*a_i + beta_2*l_i + beta_3*adjusted_sum_a_j + beta_4*adjusted_sum_l_j
    return -(1/(2*sigma2_y)) * (y_i - 2*mu_y_i)

def theta_ij(w_ij_y, params):
    theta = params[-1]
    return w_ij_y * theta

def U_i(data_i, params, y_i=None):
    # we allow the caller to pass in y_i separate from data_i
    # so that when evaluating the denominator it is easier.
    if y_i == None:
        y_i = data_i['y_i']
    G_i_output = G_i(y_i=y_i, 
                      a_i=data_i['a_i'], 
                      l_i=data_i['l_i'], 
                      adjusted_sum_a_j=data_i['adjusted_sum_a_j'], 
                      adjusted_sum_l_j=data_i['adjusted_sum_l_j'], 
                      params=params)
    
    second_term = 0
    for y_j in data_i['y_j_dict'].values():
        second_term += y_i * y_j * theta_ij(data_i['w_ij_yw_ij_theta'], params)
    
    return y_i * G_i_output + second_term

def f_Y_i_given_stuff(data_i, params):
    numerator = math.exp(U_i(data_i, params))
    
    # define a lambda function for what we want to integrate over
    integrand = lambda y_i: math.exp(U_i(data_i, params, y_i))
    
    # perform numerical integration over the range of y_i
    range_min, range_max = -np.inf, np.inf 
    denominator, abserr = quad(integrand, range_min, range_max)
    
    return numerator / denominator

def neg_log_coding_likelihood(params, Y_est_data):
    log_likelihoods = Y_est_data.apply(lambda row: np.log(f_Y_i_given_stuff(row, params)), axis=1)
    print(-np.sum(log_likelihoods))
    return -np.sum(log_likelihoods)

def optimize_params(initial_params, Y_est_data):
    result = minimize(
        fun=lambda params: neg_log_coding_likelihood(params, Y_est_data),
        x0=initial_params,
        method='L-BFGS-B'
    )
    return result.x if result.success else None

